In [1]:
from sklearn.metrics.pairwise import cosine_similarity, euclidean_distances
from  sklearn.feature_extraction.text import CountVectorizer

from scipy import sparse
import pandas as pd
import numpy as np

import seaborn as sns

In [8]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
#model = 'clean_embed_all-mpnet-base-v2.csv'
#df = pd.read_csv('/global/project/hpcg1614_shared/ca/data/banking77/{}'.format(model))
df_orig = pd.read_csv('clean.csv')

In [10]:
df_sim_results = pd.read_csv('sim_results.csv')
df_CL_results = pd.read_csv('possible_errors.csv')

In [4]:
df_ourClusters = pd.read_csv('best_e01.06.csv')

In [5]:
df_ourClusters.head()

,text,category,cluster_id
0,I am still waiting on my card?,card_arrival,31
1,What can I do if my card still hasn't arrived ...,card_arrival,31
2,I have been waiting over a week. Is the card s...,card_arrival,31
3,Can I track my card while it is in the process...,card_arrival,69
4,"How do I know if I will get my card, or if it ...",card_arrival,31


In [6]:
df_orig.head()

,text,category
0,I am still waiting on my card?,card_arrival
1,What can I do if my card still hasn't arrived ...,card_arrival
2,I have been waiting over a week. Is the card s...,card_arrival
3,Can I track my card while it is in the process...,card_arrival
4,"How do I know if I will get my card, or if it ...",card_arrival


In [11]:
df_CL_results.shape
df_CL_results.head()

(965, 6)

,Split,i,ID,Message,Truth,Maybe Better
0,1,1,5623,Why did my payment get declined?,declined_transfer,declined_card_payment
1,1,2,5803,i need a refund from you because the merchant ...,Refund_not_showing_up,request_refund
2,1,3,5012,my card was frozen due to putting in the wrong...,declined_cash_withdrawal,pin_blocked
3,1,4,2969,Can I use this app for any currency?,supported_cards_and_currencies,exchange_via_app
4,1,5,3263,My automatic top-up was unsuccessful,top_up_reverted,top_up_failed


In [12]:
df_orig['id']=df_orig.index
df_ourClusters['id']=df_ourClusters.index
df_sim_results['id']=df_sim_results['i']
df_CL_results['id']=df_CL_results['ID']

In [13]:
df_concat1 = df_orig.merge(df_ourClusters, on='id', how='left')

In [14]:
df_concat2 = df_concat1.merge(df_sim_results, on='id', how='left')

In [15]:
df_concat3 = df_concat2.merge(df_CL_results, on='id', how='left')

In [18]:
df_concat1.head()

,text_x,category_x,id,text_y,category_y,cluster_id
0,I am still waiting on my card?,card_arrival,0,I am still waiting on my card?,card_arrival,31
1,What can I do if my card still hasn't arrived ...,card_arrival,1,What can I do if my card still hasn't arrived ...,card_arrival,31
2,I have been waiting over a week. Is the card s...,card_arrival,2,I have been waiting over a week. Is the card s...,card_arrival,31
3,Can I track my card while it is in the process...,card_arrival,3,Can I track my card while it is in the process...,card_arrival,69
4,"How do I know if I will get my card, or if it ...",card_arrival,4,"How do I know if I will get my card, or if it ...",card_arrival,31


In [16]:
df_concat3.to_csv('analysis_concat.csv', index=False)

In [26]:
df_sim_results[df_sim_results['id']==75]

,sim,dist,i,message i,label i,j,message j,label j,id
18,0.960852,0.279814,75,How long will it take for me to get my card?,card_arrival,1064,How soon will I get my card?,card_delivery_estimate,75
63,0.940010,0.346381,75,How long will it take for me to get my card?,card_arrival,1087,How long does it take to receive my card?,card_delivery_estimate,75
115,0.926080,0.384499,75,How long will it take for me to get my card?,card_arrival,1104,When can I expect to receive my card?,card_delivery_estimate,75
121,0.924931,0.387475,75,How long will it take for me to get my card?,card_arrival,1103,how long does it take to get my card i am in a...,card_delivery_estimate,75
131,0.923646,0.390780,75,How long will it take for me to get my card?,card_arrival,1013,When should I expect to receive my card?,card_delivery_estimate,75
162,0.918667,0.403318,75,How long will it take for me to get my card?,card_arrival,1072,"I haven't received my card yet, how long does ...",card_delivery_estimate,75
249,0.907880,0.429232,75,How long will it take for me to get my card?,card_arrival,1045,how long should I expect to wait for my card t...,card_delivery_estimate,75
263,0.906557,0.432303,75,How long will it take for me to get my card?,card_arrival,1011,How long is the wait for my card?,card_delivery_estimate,75
299,0.903574,0.439150,75,How long will it take for me to get my card?,card_arrival,1098,When will I receive my card?,card_delivery_estimate,75
443,0.891321,0.466217,75,How long will it take for me to get my card?,card_arrival,1053,How long will it take to have my card mailed t...,card_delivery_estimate,75
